### The True State

In order to help users understand what is going on, it is necessary to be able to pull out the true state of the network at any time. This is obtained by calling the get_agent_state method and passing in 'True'. Since this observation is huge, we will focus on 'User0'.

In [1]:
import random
import inspect
from os.path import dirname
from pprint import pprint
from os.path import dirname
from CybORG import CybORG
from CybORG.Agents import B_lineAgent
from CybORG.Simulator.Scenarios import FileReaderScenarioGenerator

path = inspect.getfile(CybORG)
path = dirname(path) + f'/Simulator/Scenarios/scenario_files/Scenario1b.yaml'
sg = FileReaderScenarioGenerator(path)
env = CybORG(scenario_generator=sg, agents={'Red':B_lineAgent()})

results = env.reset(agent='Blue')

true_state = env.get_agent_state('True')
pprint(true_state['User0'])
print(76*'-')

{'Interface': [{'IP Address': IPv4Address('10.0.161.163'),
                'Interface Name': 'eth0',
                'Subnet': IPv4Network('10.0.161.160/28')}],
 'Processes': [{'Connections': [{'Transport Protocol': <TransportProtocol.UNKNOWN: 1>,
                                 'local_address': IPv4Address('0.0.0.0'),
                                 'local_port': 22}],
                'Known Path': <Path.UNKNOWN: 1>,
                'Known Process': <ProcessName.SSHD: 7>,
                'PID': 3368,
                'PPID': 5956,
                'Path': 'C:\\Program Files\\OpenSSH\\usr\\sbin',
                'Process Name': 'sshd.exe',
                'Process Type': <ProcessType.SSH: 2>,
                'Username': 'sshd_server'},
               {'Connections': [{'Transport Protocol': <TransportProtocol.UNKNOWN: 1>,
                                 'local_address': IPv4Address('0.0.0.0'),
                                 'local_port': 21}],
                'Known Path': <Path.UNKN

/home/david/anaconda3/envs/ai/lib/python3.9/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


In order to better visualise this, we will import the true_obs_to_table function, which converts this observation into a human-readable table.

In [2]:
from CybORG.Agents.Wrappers.TrueTableWrapper import true_obs_to_table

true_table = true_obs_to_table(true_state,env)
print(true_table)

Scanned column likely inaccurate.
+-----------------+--------------+-------------+-------+---------+------------+
|      Subnet     |  IP Address  |   Hostname  | Known | Scanned |   Access   |
+-----------------+--------------+-------------+-------+---------+------------+
|  10.0.201.32/28 | 10.0.201.34  |   Defender  | False |  False  |    None    |
|  10.0.201.32/28 | 10.0.201.43  | Enterprise0 | False |  False  |    None    |
|  10.0.201.32/28 | 10.0.201.42  | Enterprise1 | False |  False  |    None    |
|  10.0.201.32/28 | 10.0.201.44  | Enterprise2 | False |  False  |    None    |
| 10.0.136.192/28 | 10.0.136.198 |   Op_Host0  | False |  False  |    None    |
| 10.0.136.192/28 | 10.0.136.204 |   Op_Host1  | False |  False  |    None    |
| 10.0.136.192/28 | 10.0.136.196 |   Op_Host2  | False |  False  |    None    |
| 10.0.136.192/28 | 10.0.136.205 |  Op_Server0 | False |  False  |    None    |
| 10.0.161.160/28 | 10.0.161.163 |    User0    |  True |  False  | Privileged |
| 10.0

As red moves throughout the network, we can thus see its progress. Note how the 'User1' exploit gets root immediately.

In [3]:
for i in range(3):
    env.step()
    true_state = env.get_agent_state('True')
    true_table = true_obs_to_table(true_state,env)
    print(env.get_last_action('Red'))
    print(true_table)
    print(76*'-')

Scanned column likely inaccurate.
DiscoverRemoteSystems 10.0.161.160/28
+-----------------+--------------+-------------+-------+---------+------------+
|      Subnet     |  IP Address  |   Hostname  | Known | Scanned |   Access   |
+-----------------+--------------+-------------+-------+---------+------------+
|  10.0.201.32/28 | 10.0.201.34  |   Defender  | False |  False  |    None    |
|  10.0.201.32/28 | 10.0.201.43  | Enterprise0 | False |  False  |    None    |
|  10.0.201.32/28 | 10.0.201.42  | Enterprise1 | False |  False  |    None    |
|  10.0.201.32/28 | 10.0.201.44  | Enterprise2 | False |  False  |    None    |
| 10.0.136.192/28 | 10.0.136.198 |   Op_Host0  | False |  False  |    None    |
| 10.0.136.192/28 | 10.0.136.204 |   Op_Host1  | False |  False  |    None    |
| 10.0.136.192/28 | 10.0.136.196 |   Op_Host2  | False |  False  |    None    |
| 10.0.136.192/28 | 10.0.136.205 |  Op_Server0 | False |  False  |    None    |
| 10.0.161.160/28 | 10.0.161.163 |    User0    |

Blue team then restores 'User1' and we can see Red's access is gone.

In [4]:
from CybORG.Simulator.Actions import Restore
action = Restore(hostname='User1',session=0,agent='Blue')
env.step(action=action,agent='Blue')

true_state = env.get_agent_state('True')
true_table = true_obs_to_table(true_state,env)
print(true_table)

Scanned column likely inaccurate.
+-----------------+--------------+-------------+-------+---------+------------+
|      Subnet     |  IP Address  |   Hostname  | Known | Scanned |   Access   |
+-----------------+--------------+-------------+-------+---------+------------+
|  10.0.201.32/28 | 10.0.201.34  |   Defender  | False |  False  |    None    |
|  10.0.201.32/28 | 10.0.201.43  | Enterprise0 | False |  False  |    None    |
|  10.0.201.32/28 | 10.0.201.42  | Enterprise1 | False |  False  |    None    |
|  10.0.201.32/28 | 10.0.201.44  | Enterprise2 | False |  False  |    None    |
| 10.0.136.192/28 | 10.0.136.198 |   Op_Host0  | False |  False  |    None    |
| 10.0.136.192/28 | 10.0.136.204 |   Op_Host1  | False |  False  |    None    |
| 10.0.136.192/28 | 10.0.136.196 |   Op_Host2  | False |  False  |    None    |
| 10.0.136.192/28 | 10.0.136.205 |  Op_Server0 | False |  False  |    None    |
| 10.0.161.160/28 | 10.0.161.163 |    User0    |  True |  False  | Privileged |
| 10.0

## Other Debugging Tools

CybORG has a host of other tools to help understand the agent state. We have already see the get_observation method.

In [5]:
env.reset()
env.step()

red_obs = env.get_observation('Red')
pprint(red_obs)

blue_obs = env.get_observation('Blue')
pprint(blue_obs)
print(76*'-')

{'10.0.34.101': {'Interface': [{'IP Address': IPv4Address('10.0.34.101'),
                                'Subnet': IPv4Network('10.0.34.96/28')}]},
 '10.0.34.102': {'Interface': [{'IP Address': IPv4Address('10.0.34.102'),
                                'Subnet': IPv4Network('10.0.34.96/28')}]},
 '10.0.34.105': {'Interface': [{'IP Address': IPv4Address('10.0.34.105'),
                                'Subnet': IPv4Network('10.0.34.96/28')}]},
 '10.0.34.109': {'Interface': [{'IP Address': IPv4Address('10.0.34.109'),
                                'Subnet': IPv4Network('10.0.34.96/28')}]},
 '10.0.34.97': {'Interface': [{'IP Address': IPv4Address('10.0.34.97'),
                               'Subnet': IPv4Network('10.0.34.96/28')}]},
 'success': <TrinaryEnum.TRUE: 1>}
{'success': <TrinaryEnum.UNKNOWN: 2>}
----------------------------------------------------------------------------


We have also seen the get_last_action method. Note that since we haven't specified an action or agent for blue it defaults to Sleep.

In [6]:
red_action = env.get_last_action('Red')
print(red_action)

blue_action = env.get_last_action('Blue')
print(blue_action)

DiscoverRemoteSystems 10.0.34.96/28
Sleep


The get_action_space method allows us to get the action space of any agent.

In [7]:
red_action_space = env.get_action_space('Red')
print(list(red_action_space))

blue_action_space = env.get_action_space('Blue')
print(list(blue_action_space.keys()))

['action', 'subnet', 'ip_address', 'session', 'username', 'password', 'process', 'port', 'target_session', 'agent', 'hostname']
['action', 'subnet', 'ip_address', 'session', 'username', 'password', 'process', 'port', 'target_session', 'agent', 'hostname']


The get_ip_map method allows us to see which hostnames are associated with each ip. This is something that is known to Blue, but not to Red at the beginning of the game so be careful to not abuse it.

In [8]:
env.get_ip_map()

{'Defender': IPv4Address('10.0.90.87'),
 'Enterprise0': IPv4Address('10.0.90.91'),
 'Enterprise1': IPv4Address('10.0.90.85'),
 'Enterprise2': IPv4Address('10.0.90.88'),
 'Op_Host0': IPv4Address('10.0.65.130'),
 'Op_Host1': IPv4Address('10.0.65.137'),
 'Op_Host2': IPv4Address('10.0.65.136'),
 'Op_Server0': IPv4Address('10.0.65.138'),
 'User0': IPv4Address('10.0.34.105'),
 'User1': IPv4Address('10.0.34.102'),
 'User2': IPv4Address('10.0.34.109'),
 'User3': IPv4Address('10.0.34.97'),
 'User4': IPv4Address('10.0.34.101'),
 'Enterprise_router': IPv4Address('10.0.90.86'),
 'Operational_router': IPv4Address('10.0.65.135'),
 'User_router': IPv4Address('10.0.34.99')}

The get_rewards method allows us to see the rewards for all agents.

In [9]:
env.get_rewards()

{'Blue': {'HybridAvailabilityConfidentiality': 0.0, 'action_cost': 0},
 'Green': {'None': 0.0, 'action_cost': 0},
 'Red': {'HybridImpactPwn': 0.0, 'action_cost': 0}}

Finally, the set_seed method allows us to set a random seed.

In [10]:
env.set_seed(100)